In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-diagnostics2/intents2..json
/kaggle/input/car-diagnostics2/test.py
/kaggle/input/car-diagnostics2/main.py
/kaggle/input/car-diagnostics2/README.md
/kaggle/input/car-diagnostics2/katana-assistant-data.pkl
/kaggle/input/car-diagnostics2/app.py
/kaggle/input/car-diagnostics2/requirements.txt
/kaggle/input/car-diagnostics2/intents.json
/kaggle/input/car-diagnostics2/tempCodeRunnerFile.py
/kaggle/input/car-diagnostics2/data.pickle


In [2]:
!pip install tflearn


In [3]:
import tensorflow.compat.v1 as tf
import tensorflow  
import nltk
import json
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [4]:
with open("/kaggle/input/car-diagnostics2/intents.json") as file:
    data = json.load(file)
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))# sort classes
classes = sorted(list(set(classes)))# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)            

77 documents
20 classes ['Low engine oil level', 'Low fuel mileage', 'age', 'alignment', 'alternator', 'battery', 'engine overheating', 'engine_start', 'exhaust smoke', 'fuel_system', 'goodbye', 'greeting', 'greetings', 'help', 'hours', 'low ac cooling', 'squealing brakes', 'starter_motor', 'starting issue', 'suspension']
122 unique stemmed words ["'s", 'a', 'ac', 'ag', 'air', 'altern', 'am', 'anym', 'ar', 'bad', 'battery', 'becaus', 'blow', 'brak', 'brandon', 'burn', 'but', 'bye', 'cao', 'car', 'charg', 'click', 'cold', 'com', 'crank', 'cya', 'day', 'dead', 'died', 'dim', 'do', 'doesnt', 'dont', 'driv', 'engin', 'exhaust', 'freeway', 'front', 'fuel', 'funny', 'get', 'going', 'good', 'goodby', 'greet', 'guy', 'hav', 'headlight', 'hello', 'help', 'hey', 'hi', 'hold', 'hot', 'hour', 'how', 'i', 'in', 'is', 'isnt', 'issu', 'it', 'keep', 'know', 'lat', 'leak', 'leav', 'left', 'level', 'light', 'lot', 'low', 'mak', 'misfir', 'mor', 'mot', 'my', "n't", 'nee', 'nois', 'not', 'of', 'oil', 'old

In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [7]:
# Compile model. 
# Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd= SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [8]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
16/16 [==============================] - 1s 3ms/step - loss: 3.0567 - accuracy: 0.0260
Epoch 2/200
16/16 [==============================] - 0s 2ms/step - loss: 2.9712 - accuracy: 0.1169
Epoch 3/200
16/16 [==============================] - 0s 2ms/step - loss: 2.8485 - accuracy: 0.1948
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 2.8194 - accuracy: 0.2208
Epoch 5/200
16/16 [==============================] - 0s 2ms/step - loss: 2.6741 - accuracy: 0.2468
Epoch 6/200
16/16 [==============================] - 0s 3ms/step - loss: 2.5871 - accuracy: 0.3247
Epoch 7/200
16/16 [==============================] - 0s 2ms/step - loss: 2.5194 - accuracy: 0.2987
Epoch 8/200
16/16 [==============================] - 0s 2ms/step - loss: 2.3398 - accuracy: 0.2857
Epoch 9/200
16/16 [==============================] - 0s 2ms/step - loss: 2.2839 - accuracy: 0.2987
Epoch 10/200
16/16 [==============================] - 0s 2ms/step - loss: 2.1708 - accuracy: 0.3377
Epoch 11/

In [13]:
with open('car-diagnostic.pkl', 'wb') as f:
    pickle.dump(model, f)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-22 12:41:22         2335
variables.h5                                   2023-03-22 12:41:22       122808
metadata.json                                  2023-03-22 12:41:22           64


In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [11]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [12]:
classify_local("Car Brakes don't work")

found in bag: car
found in bag: brak
found in bag: do
found in bag: n't
found in bag: work
1/1 [==============================] - 0s 109ms/step


[('squealing brakes', '0.35313186')]